# Meta update learning attempts to use transfer learning

It's a significant refactor and review to go over this code and remember what exactly everything is doing, but it would be nice to get this running for ACMBC at least

Need to regenerate a dataset that fits the hyperparameters of the original paper here.
https://arxiv.org/abs/2003.06113


## Imports

In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import tensorflow as tf
# tf.enable_eager_execution()
# tf.enable_v2_behavior()
# sess = tf.InteractiveSession()
# sess.as_default()

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
# reload(psce)
from keras.utils import multi_gpu_model
import keras.optimizers
import ensembleReader as er
reload(er)
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss
from functools import lru_cache

Using TensorFlow backend.


In [2]:
eds = er.EdfDatasetSegments(pre_cooldown=0, post_cooldown=0, sample_time=0, num_seconds=0.2, n_process=16)
train_label_files_segs = eds.get_train_split()
valid_label_files_segs = eds.get_valid_split()
test_label_files_segs = eds.get_test_split()

Starting 16 processes
retrieving: 0
Starting 16 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
retrieving: 3000
starting custom_annotater
Starting 20 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
starting custom_annotater
Starting 20 processes
retrieving: 0
starting custom_annotater
Starting 20 processes
retrieving: 0


In [3]:
train_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=train_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=400/250*pd.Timedelta(seconds=1), shuffle=False)
valid_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=valid_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=400/250*pd.Timedelta(seconds=1),  shuffle=False)
test_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=test_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=400/250*pd.Timedelta(seconds=1),  shuffle=False)

In [8]:
len(train_edss), len(valid_edss), len(test_edss)

(7204971, 2003340, 2665266)

In [5]:
import pickle as pkl

In [6]:
train_edss.n_process = 16
valid_edss.n_process = 16
test_edss.n_process = 16

In [10]:
train_edss[0]

(array([[  98.16088631,   75.46726397,   21.82835267, ...,   44.39217343,
           93.44785785,  -15.81407579],
        [ 681.78663633,  522.53736893,  151.83168032, ...,  309.0285616 ,
          646.15079753, -109.95548131],
        [2115.28297853, 1611.58586504,  470.54208614, ...,  958.32733831,
         1990.91343858, -340.67129047],
        ...,
        [-548.13594559, -242.21613638,  -71.86730377, ..., -145.93090278,
         -299.19665001,   50.85505304],
        [-544.65301041, -239.5143143 ,  -71.05878124, ..., -144.30710149,
         -295.90415088,   50.36805415],
        [-540.80219081, -236.56886298,  -70.19198778, ..., -142.5510337 ,
         -292.31267112,   49.8586534 ]]),
 (False,
  'bckg',
  '/home/ms994/v1.5.0/edf/train/01_tcp_ar/103/00010301/s001_2013_05_20/00010301_s001_t000.edf'))

In [9]:
for i in range(73):
    train_edss_subsample = train_edss[i*100000:min((i+1)*100000, len(train_edss))]
    pkl.dump(train_edss_subsample, open(f"/n/scratch3/users/m/ms994/train_mups_{i}.pkl","wb"))
    del train_edss_subsample

Starting 16 processes
retrieving: 0
retrieving: 250
retrieving: 500
retrieving: 750
retrieving: 1000
retrieving: 1250
retrieving: 1500
retrieving: 1750
retrieving: 2000
retrieving: 2250
retrieving: 2500
retrieving: 2750
retrieving: 3000
retrieving: 3250
retrieving: 3500
retrieving: 3750
retrieving: 4000
retrieving: 4250
retrieving: 4500
retrieving: 4750
retrieving: 5000
retrieving: 5250
retrieving: 5500
retrieving: 5750
retrieving: 6000
retrieving: 6250
retrieving: 6500
retrieving: 6750
retrieving: 7000
retrieving: 7250
retrieving: 7500
retrieving: 7750
retrieving: 8000
retrieving: 8250
retrieving: 8500
retrieving: 8750
retrieving: 9000
retrieving: 9250
retrieving: 9500
retrieving: 9750
retrieving: 10000
retrieving: 10250
retrieving: 10500
retrieving: 10750
retrieving: 11000
retrieving: 11250
retrieving: 11500
retrieving: 11750
retrieving: 12000
retrieving: 12250
retrieving: 12500
retrieving: 12750
retrieving: 13000
retrieving: 13250
retrieving: 13500
retrieving: 13750
retrieving: 1400

retrieving: 113250
retrieving: 113500
retrieving: 113750
retrieving: 114000
retrieving: 114250
retrieving: 114500
retrieving: 114750
retrieving: 115000
retrieving: 115250
retrieving: 115500
retrieving: 115750
retrieving: 116000
retrieving: 116250
retrieving: 116500
retrieving: 116750
retrieving: 117000
retrieving: 117250
retrieving: 117500
retrieving: 117750
retrieving: 118000
retrieving: 118250
retrieving: 118500
retrieving: 118750
retrieving: 119000
retrieving: 119250
retrieving: 119500
retrieving: 119750
retrieving: 120000
retrieving: 120250
retrieving: 120500
retrieving: 120750
retrieving: 121000
retrieving: 121250
retrieving: 121500
retrieving: 121750
retrieving: 122000
retrieving: 122250
retrieving: 122500
retrieving: 122750
retrieving: 123000
retrieving: 123250
retrieving: 123500
retrieving: 123750
retrieving: 124000
retrieving: 124250
retrieving: 124500
retrieving: 124750
retrieving: 125000
retrieving: 125250
retrieving: 125500
retrieving: 125750
retrieving: 126000
retrieving: 

retrieving: 225250
retrieving: 225500
retrieving: 225750
retrieving: 226000
retrieving: 226250
retrieving: 226500
retrieving: 226750
retrieving: 227000
retrieving: 227250
retrieving: 227500
retrieving: 227750
retrieving: 228000
retrieving: 228250
retrieving: 228500
retrieving: 228750
retrieving: 229000
retrieving: 229250
retrieving: 229500
retrieving: 229750
retrieving: 230000
retrieving: 230250
retrieving: 230500
retrieving: 230750
retrieving: 231000
retrieving: 231250
retrieving: 231500
retrieving: 231750
retrieving: 232000
retrieving: 232250
retrieving: 232500
retrieving: 232750
retrieving: 233000
retrieving: 233250
retrieving: 233500
retrieving: 233750
retrieving: 234000
retrieving: 234250
retrieving: 234500
retrieving: 234750
retrieving: 235000
retrieving: 235250
retrieving: 235500
retrieving: 235750
retrieving: 236000
retrieving: 236250
retrieving: 236500
retrieving: 236750
retrieving: 237000
retrieving: 237250
retrieving: 237500
retrieving: 237750
retrieving: 238000
retrieving: 

retrieving: 333000
retrieving: 333250
retrieving: 333500
retrieving: 333750
retrieving: 334000
retrieving: 334250
retrieving: 334500
retrieving: 334750
retrieving: 335000
retrieving: 335250
retrieving: 335500
retrieving: 335750
retrieving: 336000
retrieving: 336250
retrieving: 336500
retrieving: 336750
retrieving: 337000
retrieving: 337250
retrieving: 337500
retrieving: 337750
retrieving: 338000
retrieving: 338250
retrieving: 338500
retrieving: 338750
retrieving: 339000
retrieving: 339250
retrieving: 339500
retrieving: 339750
retrieving: 340000
retrieving: 340250
retrieving: 340500
retrieving: 340750
retrieving: 341000
retrieving: 341250
retrieving: 341500
retrieving: 341750
retrieving: 342000
retrieving: 342250
retrieving: 342500
retrieving: 342750
retrieving: 343000
retrieving: 343250
retrieving: 343500
retrieving: 343750
retrieving: 344000
retrieving: 344250
retrieving: 344500
retrieving: 344750
retrieving: 345000
retrieving: 345250
retrieving: 345500
retrieving: 345750
retrieving: 

retrieving: 440750
retrieving: 441000
retrieving: 441250
retrieving: 441500
retrieving: 441750
retrieving: 442000
retrieving: 442250
retrieving: 442500
retrieving: 442750
retrieving: 443000
retrieving: 443250
retrieving: 443500
retrieving: 443750
retrieving: 444000
retrieving: 444250
retrieving: 444500
retrieving: 444750
retrieving: 445000
retrieving: 445250
retrieving: 445500
retrieving: 445750
retrieving: 446000
retrieving: 446250
retrieving: 446500
retrieving: 446750
retrieving: 447000
retrieving: 447250
retrieving: 447500
retrieving: 447750
retrieving: 448000
retrieving: 448250
retrieving: 448500
retrieving: 448750
retrieving: 449000
retrieving: 449250
retrieving: 449500
retrieving: 449750
retrieving: 450000
retrieving: 450250
retrieving: 450500
retrieving: 450750
retrieving: 451000
retrieving: 451250
retrieving: 451500
retrieving: 451750
retrieving: 452000
retrieving: 452250
retrieving: 452500
retrieving: 452750
retrieving: 453000
retrieving: 453250
retrieving: 453500
retrieving: 

Process Process-181:
Process Process-169:
Process Process-178:
Process Process-175:
Process Process-170:
Process Process-168:
Process Process-174:
Process Process-173:
Process Process-180:
Process Process-172:
Traceback (most recent call last):
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ms994/dbmi_eeg_clustering/util_funcs.py", line 146, in helper_process
    out_q.put((i, self[i]))
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/series.py", line 268, in __init__
    self.name = name
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/reshape/concat.py", line 448, in _get_new_axes
    new_axes[i] = self._get_comb_axis(i)
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
KeyboardInterrupt
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
  File "/home/ms994/dbmi_eeg_clustering/data_reader.py", line 675, in edf_eeg_2_df
 

KeyboardInterrupt: 

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/indexes/api.py", line 117, in _get_combined_index
    index = _union_indexes(indexes, sort=sort)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/generic.py", line 5079, in __setattr__
    object.__getattribute__(self, name)
KeyboardInterrupt
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/reshape/concat.py", line 403, in get_result
    df = cons(data, index=index)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/apply.py", line 257, in apply_standard
    self.apply_series_generator()
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py", line 448, in _generate_range
    values, _tz = generate_regular_range(start, end, periods, freq)
  File "/home/ms994/dbmi_eeg_clustering/data_reader.py", line 675, in edf_eeg_2_df

  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/frame.py", line 6487, in apply
    return op.get_result()
Process Process-176:
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/apply.py", line 151, in get_result
    return self.apply_standard()
  File "/home/ms994/dbmi_eeg_clustering/util_funcs.py", line 146, in helper_process
    out_q.put((i, self[i]))
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/apply.py", line 257, in apply_standard
    self.apply_series_generator()
  File "/home/ms994/dbmi_eeg_clustering/ensembleReader.py", line 515, in __getitem__
    max_length=self.gap)
  File "/home/ms994/miniconda3/envs/keras-redo-cpu/lib/python3.7/site-packages/pandas/core/apply.py", line 285, in apply_series_g

In [ ]:
for i in range(21):
    valid_edss_subsample = valid_edss[i*100000:min((i+1)*100000, len(valid_edss))]
    pkl.dump(valid_edss_subsample, open(f"/n/scratch3/users/m/ms994/valid_mups_{i}.pkl","wb"))
    del valid_edss_subsample

In [ ]:
for i in range(27):
    test_edss_subsample = test_edss[i*100000:min((i+1)*100000, len(test_edss))]
    pkl.dump(test_edss_subsample, open(f"/n/scratch3/users/m/ms994/test_mups_{i}.pkl","wb"))
    del test_edss_subsample

In [ ]:
%%bash
scancel 9781732

In [ ]:
plt.plot(train_edss[1][0])

In [ ]:
len(train_edss)

In [ ]:
train_edss[0][0].shape